# DSA 8420 Project 1
#### Liv Gamble & Alexander Harriman

## Farmer Crop Management

### Q1: An Abstract Model

In [11]:
import pyomo.environ as pyo

In [12]:
# Create an Abstract model
model = pyo.AbstractModel()

In [13]:
# Build the index set
model.Crop = pyo.Set()

In [14]:
# Build parameters
model.w_labor = pyo.Param(model.Crop,within=pyo.NonNegativeReals)
model.w_capital = pyo.Param(model.Crop,within=pyo.NonNegativeReals)
model.profit = pyo.Param(model.Crop,within=pyo.NonNegativeReals)
model.avail_land = pyo.Param(within=pyo.NonNegativeReals)
model.avail_labor = pyo.Param(within=pyo.NonNegativeReals)
model.avail_capital = pyo.Param(within=pyo.NonNegativeReals)

In [15]:
# Build the decision variables
model.x = pyo.Var(model.Crop, within=pyo.NonNegativeReals) 

In [16]:
# Build the objective function
def obj_value_rule(model):
    return sum(model.profit[i]*model.x[i] for i in model.Crop)

model.obj = pyo.Objective(rule=obj_value_rule, sense = pyo.maximize)

In [17]:
# Build the land constraint
def land_rule(model):
    return sum(model.x[i] for i in model.Crop) <= model.avail_land

model.land = pyo.Constraint(rule=land_rule)

In [18]:
# Build the labor constraint
def labor_rule(model):
    return sum(model.w_labor[i]*model.x[i] for i in model.Crop) <= model.avail_labor

model.labor = pyo.Constraint(rule=labor_rule)

In [19]:
# Build the capital constraint
def capital_rule(model):
    return sum(model.w_capital[i]*model.x[i] for i in model.Crop) <= model.avail_capital

model.capital = pyo.Constraint(rule=capital_rule)

In [20]:
# Load data
instance = model.create_instance('agriculture.dat')

In [21]:
# Solve the model through the GLPK solver
solver = pyo.SolverFactory('glpk')
solver.solve(instance)

{'Problem': [{'Name': 'unknown', 'Lower bound': 360.0, 'Upper bound': 360.0, 'Number of objectives': 1, 'Number of constraints': 4, 'Number of variables': 4, 'Number of nonzeros': 10, 'Sense': 'maximize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 0, 'Number of created subproblems': 0}}, 'Error rc': 0, 'Time': 0.09763503074645996}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [22]:
# Print the results
instance.display()
print()
print('Optimal objective value: ', pyo.value(instance.obj))

Model unknown

  Variables:
    x : Size=3, Index=Crop
        Key      : Lower : Value : Upper : Fixed : Stale : Domain
            Corn :     0 :   6.0 :  None : False : False : NonNegativeReals
            Oats :     0 :   6.0 :  None : False : False : NonNegativeReals
        Soybeans :     0 :   0.0 :  None : False : False : NonNegativeReals

  Objectives:
    obj : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 360.0

  Constraints:
    land : Size=1
        Key  : Lower : Body : Upper
        None :  None : 12.0 :  12.0
    labor : Size=1
        Key  : Lower : Body : Upper
        None :  None : 48.0 :  48.0
    capital : Size=1
        Key  : Lower : Body  : Upper
        None :  None : 324.0 : 360.0

Optimal objective value:  360.0


In [23]:
# Print the optimal solution
for i in instance.Crop: 
    print(i, pyo.value(instance.x[i]))

Corn 6.0
Soybeans 0.0
Oats 6.0


### Q4: An Objective Function for the Dual

In [24]:
import pyomo.environ as pyo

In [25]:
mod = pyo.ConcreteModel()

In [26]:
#Decision Variables
mod.x = pyo.Var({1,2,3}) 

In [27]:
#Objective Function
mod.obj = pyo.Objective(expr = 12*mod.x[1]+48*mod.x[2]+360*mod.x[3])

In [28]:
#Constraints
mod.con1 = pyo.Constraint(expr = mod.x[1]+6*mod.x[2]+36*mod.x[3] >= 40)
mod.con2 = pyo.Constraint(expr = mod.x[1]+6*mod.x[2]+24*mod.x[3] >= 30)
mod.con3 = pyo.Constraint(expr = mod.x[1]+2*mod.x[2]+18*mod.x[3] >= 20)
mod.con4 = pyo.Constraint(expr = mod.x[1] >= 0)
mod.con5 = pyo.Constraint(expr = mod.x[2] >= 0)
mod.con6 = pyo.Constraint(expr = mod.x[3] >= 0)

In [29]:
mod.pprint()

1 Set Declarations
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {1, 2, 3}

1 Var Declarations
    x : Size=3, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :  None :  None :  None : False :  True :  Reals
          2 :  None :  None :  None : False :  True :  Reals
          3 :  None :  None :  None : False :  True :  Reals

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : 12*x[1] + 48*x[2] + 360*x[3]

6 Constraint Declarations
    con1 : Size=1, Index=None, Active=True
        Key  : Lower : Body                    : Upper : Active
        None :  40.0 : x[1] + 6*x[2] + 36*x[3] :  +Inf :   True
    con2 : Size=1, Index=None, Active=True
        Key  : Lower : Body                    : Upper : Active
        None :  30.0 : x[1] + 6*x[2] + 24*x[3] :  +Inf :   T

In [30]:
#Solving
solver = pyo.SolverFactory('glpk')
solver.solve(mod)

{'Problem': [{'Name': 'unknown', 'Lower bound': 360.0, 'Upper bound': 360.0, 'Number of objectives': 1, 'Number of constraints': 7, 'Number of variables': 4, 'Number of nonzeros': 13, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 0, 'Number of created subproblems': 0}}, 'Error rc': 0, 'Time': 0.11449599266052246}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [31]:
#Results
print()
mod.display()
print()
print('Optimal value: ', pyo.value(mod.obj))


Model unknown

  Variables:
    x : Size=3, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :  None :  10.0 :  None : False : False :  Reals
          2 :  None :   5.0 :  None : False : False :  Reals
          3 :  None :   0.0 :  None : False : False :  Reals

  Objectives:
    obj : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 360.0

  Constraints:
    con1 : Size=1
        Key  : Lower : Body : Upper
        None :  40.0 : 40.0 :  None
    con2 : Size=1
        Key  : Lower : Body : Upper
        None :  30.0 : 40.0 :  None
    con3 : Size=1
        Key  : Lower : Body : Upper
        None :  20.0 : 20.0 :  None
    con4 : Size=1
        Key  : Lower : Body : Upper
        None :   0.0 : 10.0 :  None
    con5 : Size=1
        Key  : Lower : Body : Upper
        None :   0.0 :  5.0 :  None
    con6 : Size=1
        Key  : Lower : Body : Upper
        None :   0.0 :  0.0 :  None

Optimal value:  360.0
